In [11]:
import requests
import json


In [17]:
def getToken(username, password):
    res = requests.post(
        "http://localhost:8000/users/auth/",
        data={"username": username, "password": password},
        verify=False,
    )
    return res.json()["token"]


def makeRoom(token, title, subtitle=None):
    res = requests.post(
        "http://localhost:8000/rooms/",
        headers={"Authorization": "Token " + token},
        data={"subtitle": subtitle, "title": title},
    )
    return res


def makeEvent(token, room, event_type, args, parent_event=None):
    data = {"room": room, "event_type":event_type, "args":args}
    if parent_event:
        data["parent_event"] = parent_event
    res = requests.post(
        "http://localhost:8000/events/",
        headers={"Authorization": "Token " + token, "content-type": "application/json"},
        data=json.dumps(data),
    )
    return res

def makeTune(token):
    with open("api for extra args.txt", 'rb') as fobj:
        res = requests.post(
            "http://localhost:8000/tunes/",
            headers={"Authorization": "Token " + token},
            files={'file': fobj},
        )
        return res
    
def getEvents(token, room, query):
    url =  "http://localhost:8000/rooms/{0}/events/?{1}".format(room, query)
    
    res = requests.get(url, 
                       headers={"Authorization": "Token " + token})
    return res

def getTuneMeta(token, tune_id):
    url = "http://localhost:8000/tunes/{}/meta/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def getTuneData(token, tune_id):
    url = "http://localhost:8000/tunes/{}/data/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res




def makeUser(username, password):
    url = "http://localhost:8000/users/"
    res = requests.post(url,
        data={"username": username, "password": password},
    )
    return res

def getTuneSync(token, room_id):
    url = "http://localhost:8000/rooms/{}/tunesync/".format(room_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def kickUser(token, room_id, kicked_user):
    return makeEvent(token, room_id, "U", {"type": "K", "user": kicked_user})

def inviteUsers(token, room_id, users):
    return makeEvent(token, room_id, "U", {"type": "I", "users": users})

def changeUser(token, room_id, user, role):
    return makeEvent(token, room_id, "U", {"type": "C", "user": user, "role": role})

def userJoin(token, room_id, is_accepted):
    return makeEvent(token, room_id, "U", {"type": "J", "is_accepted": is_accepted})

def getTune(token, query):
    url = "http://localhost:8000/tunes?{}".format(query)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res


def getRoomEvent(token, room_id):
    url = "http://localhost:8000/rooms/{}/events/".format(room_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def getRoomTuneSync(token, room_id):
    url = "http://localhost:8000/rooms/{}/tunesync/".format(room_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res


def getUsers(token, page, query):
    url = "http://localhost:8000/users/?page={}&{}".format(page, query)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def getPoll(token, room):
    url = "http://localhost:8000/rooms/{}/polls".format(room)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

    

In [124]:
res = inviteUsers(TOKEN, 1, [5])
res.json()

{'details': 'cannot invite others to personal room'}

In [110]:
res = getEvents(TOKEN, 4, 'parent_event=M')
res.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /rooms/4/events/?parent_event=M (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1be9a53630>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [5]:
res = getUsers(TOKEN,1, "username__icontains=o")
res.json()

{'count': 2,
 'next': None,
 'previous': None,
 'results': [{'id': 4, 'username': 'goku'},
  {'id': 5, 'username': 'krqieoeoaxreweazaqecqlailin'}]}

In [118]:
res = getTune(TOKEN, "uploader_username=goeku")
res.json()

{'count': 9,
 'next': None,
 'previous': None,
 'results': [{'id': 1,
   'uploader': 2,
   'name': "['Cross Examination ~ Moderate 2001']",
   'artist': "['Ace Attorney']",
   'album': "['(AA1) Phoenix Wright - Ace Attorney OST']",
   'mime': 'audio/mp3',
   'length': 104.70515},
  {'id': 2,
   'uploader': 2,
   'name': "['Logic And Trick']",
   'artist': "['Ace Attorney']",
   'album': "['(AA1) Phoenix Wright - Ace Attorney OST']",
   'mime': 'audio/mp3',
   'length': 152.457},
  {'id': 3,
   'uploader': 2,
   'name': "['Phoenix Wright ~ Objection! 2001']",
   'artist': "['Ace Attorney']",
   'album': "['(AA1) Phoenix Wright - Ace Attorney OST']",
   'mime': 'audio/mp3',
   'length': 73.3321},
  {'id': 4,
   'uploader': 2,
   'name': "['Cross Examination ~ Allegro 2001']",
   'artist': "['Ace Attorney']",
   'album': "['(AA1) Phoenix Wright - Ace Attorney OST']",
   'mime': 'audio/mp3',
   'length': 126.88315},
  {'id': 5,
   'uploader': 2,
   'name': "['Pursuit ~ Cornered']",
   'art

In [100]:
res = getRoomTuneSync(TOKEN, 2)
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [15]:
res = getRoomEvent(TOKEN, 3)
res.json()

{'count': 2,
 'next': None,
 'previous': None,
 'results': [{'args': {'content': 'er'},
   'parent_event_id': None,
   'creation_time': '2020-04-06T17:40:23.033991Z',
   'event_type': 'M',
   'username': 'goku',
   'event_id': 13,
   'user_id': 2,
   'room_id': 3},
  {'args': {'content': 'sex'},
   'parent_event_id': None,
   'creation_time': '2020-04-06T17:11:29.022203Z',
   'event_type': 'M',
   'username': 'goku',
   'event_id': 1,
   'user_id': 2,
   'room_id': 3}]}

In [22]:
res = getTune(TOKEN, "uploader=bbb")
res.json()

[]

In [23]:
res = makeUser("krqieoeoaxreweazaqecqlailin", "green123")
res.json()


{'token': 'ab74a4240c1eea2f1b2225a112d136514604de5c', 'user_id': 5}

In [42]:
makeUser("goku", "green123")

<Response [200]>

In [13]:
USERNAME = "goku"
PASSWORD = "green123"


TOKEN = getToken(USERNAME, PASSWORD)

In [14]:
TOKEN

'b9d3b71bcbbc64addad5ecf5179c3457580ecddf'

In [10]:
res = makeRoom(TOKEN, "Personal Room")
res.json()

{'details': "title must not be 'Personal Room'"}

In [47]:
res = makeEvent(TOKEN, 3, "M", {"content": "sex"})
res.json()

{'id': 1,
 'room': 3,
 'creation_time': '2020-04-06T17:11:29.022203Z',
 'event_type': 'M',
 'author': 2,
 'parent_event_id': None,
 'args': {'content': 'sex'},
 'isDeleted': False}

In [ ]:
# django=> select * from tunesync_membership;
#  id | state | role | room_id | user_id 
# ----+-------+------+---------+---------
#   1 | A     | A    |       1 |       3
#   2 | A     | A    |       2 |       2
# (2 rows)


In [68]:
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [12]:
res = makeEvent(TOKEN, 3, "T", {"play":{"timestamp":0.0, 'is_playing': True, 'queue_index': 1}})
res.json()

{'details': 'There are no songs in queue to play'}

In [13]:
res = makeEvent(TOKEN, 4, "T", {"modify_queue":{"queue":[1]}})
res.json()

{'last_modify_queue': {'queue': [[1,
    23.412125,
    "['Drums Urban Percussion Intro 2 (Music For Video)']"]],
  'event_id': 16},
 'last_play': None,
 'play_time': None,
 'room_id': 4}

In [44]:
res = makeEvent(TOKEN, 3, "U", {"type": [1,2]})
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [24]:
res

<Response [200]>

In [47]:
res = makeEvent(TOKEN, 8, "PL", {"song_id": 1, "is_playing":True, "timestamp": 10.4})

In [48]:
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [67]:
res = inviteUsers(TOKEN, 2, [4])
res.json()

{'id': 8,
 'room': 2,
 'creation_time': '2020-03-20T18:57:22.611842Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'I', 'users': [4]},
 'isDeleted': False}

In [47]:
res = inviteUsers(TOKEN, 3, [7])
res.json()

{'details': 'user 7 does not exist'}

In [66]:
res = kickUser(TOKEN, 2, 4)
res.json()

{'id': 6,
 'room': 2,
 'creation_time': '2020-03-20T18:57:12.740969Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'K', 'user': 4},
 'isDeleted': False}

In [46]:
res.status_code

400

In [74]:
res = changeUser(TOKEN, 2, 2, "D")
res.json()

{'id': 10,
 'room': 2,
 'creation_time': '2020-03-21T00:08:04.211899Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'C', 'user': 2, 'role': 'D'},
 'isDeleted': False}

In [69]:
res.json()

{'details': 'user is not in the room'}

In [34]:
res = makeEvent(TOKEN, 1, "M", {"content": "sex"})

In [35]:
res.json()

{'id': 9,
 'room': 1,
 'creation_time': '2020-03-16T12:40:21.851273Z',
 'event_type': 'M',
 'author': 2,
 'parent_event_id': None,
 'args': {'content': 'sex'},
 'isDeleted': False}

In [36]:
res = makeEvent(TOKEN, 8, "PO", {"action": "PL", "song_id": 1, "is_playing":True, "timestamp": 10.4})

In [37]:
res.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [67]:
res = makeEvent(TOKEN, 3, "PO", {"action": "MQ", "queue": [1]})

In [68]:
res.json()

{'poll': 14,
 'args': {'action': 'MQ', 'queue': [1]},
 'vote_percentage': 0.0,
 'agrees': 0,
 'disagrees': 0,
 'is_actve': True,
 'is_successful': False,
 'room_id': 3}

In [62]:
res = makeEvent(TOKEN, 3, "V", {"agree": True}, 2)
res.json()

{'poll': 2,
 'args': {'queue': [1], 'action': 'MQ'},
 'vote_percentage': 1.0,
 'agrees': 1,
 'disagrees': 0,
 'is_actve': True,
 'is_successful': False,
 'room_id': 3}

In [51]:
res = makeEvent(TOKEN, 8, "PO", {"action": "U", "type": "K" , "user": 4})

In [31]:
res.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [69]:
res = getPoll(TOKEN, 3)
res.json()

{'count': 2,
 'next': None,
 'previous': None,
 'results': [{'poll': 2,
   'args': {'queue': [1], 'action': 'MQ'},
   'vote_percentage': 1.0,
   'agrees': 1,
   'disagrees': 0,
   'is_actve': True,
   'is_successful': False,
   'room_id': 3},
  {'poll': 14,
   'args': {'queue': [1], 'action': 'MQ'},
   'vote_percentage': 0.0,
   'agrees': 0,
   'disagrees': 0,
   'is_actve': True,
   'is_successful': False,
   'room_id': 3}]}

In [18]:
res = makeTune(TOKEN)

In [92]:
res = getTuneMeta(TOKEN, 2)

In [15]:
res = getUser(TOKEN, 0, 5)

In [107]:
res = getTuneData(TOKEN,2)

In [19]:
res.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
import mutagen

In [9]:
audio = mutagen.File("api for extra args.txt", easy=True)
audio

In [10]:
audio

In [40]:
audio.info.length

23.412125

In [24]:
audio["title"] = None

ValueError: Invalid MultiSpec data: None

In [25]:
audio = {}

In [26]:
audio["x"] = None

In [28]:
audio["c"]

KeyError: 'c'